### 安装下载spacy和必要的库

```bash
# 确保已经安装了中文模型

pip install spacy
python -m spacy download zh_core_web_sm
```

In [29]:
import spacy

# 加载中文模型
nlp = spacy.load("zh_core_web_md")

In [30]:
# 使用nlp对象处理一段文本并生成doc实例
doc = nlp("怎么解锁宠物的家？")

# 遍历doc实例中的词符
for token in doc:
    print(token.text)

怎么
解锁
宠物
的
家
？


In [31]:
doc = nlp(u'宠物的家在哪啊？')
for token in doc:
    print(token.text, token.pos_, token.dep_)

宠物 NOUN nmod:assmod
的 PART case
家 NOUN nsubj
在 VERB dep
哪 PRON dobj
啊 PART discourse
？ PUNCT ROOT


### 词的属性

In [19]:
doc = nlp("怎么解锁宠物的家？")

print("Index:   ", [token.i for token in doc])
print("Text:    ", [token.text for token in doc])

print("is_alpha:", [token.is_alpha for token in doc])
print("is_punct:", [token.is_punct for token in doc])
print("like_num:", [token.like_num for token in doc])

Index:    [0, 1, 2, 3, 4, 5]
Text:     ['怎么', '解锁', '宠物', '的', '家', '？']
is_alpha: [True, True, True, True, True, False]
is_punct: [False, False, False, False, False, True]
like_num: [False, False, False, False, False, False]


### NLP模型训练

使spaCy可以_从语境中_抽取到语言学属性的模型

 - 词性标注
 - 依存关系解析
 - 命名实体识别

从标注过的文本中训练而来

可以用更多的标注数据来更新模型，优化抽取结果

```
python -m spacy download zh_core_web_sm
```


```python
import spacy

nlp = spacy.load("zh_core_web_sm")
```

#### 词性标注

In [21]:
import spacy

# 读取小版本的中文流程
nlp = spacy.load("zh_core_web_sm")

# 处理文本
doc = nlp("怎么解锁宠物的家？")

# 遍历词符
for token in doc:
    # Print the text and the predicted part-of-speech tag
    print(token.text, token.pos_)

怎么 ADV
解锁 VERB
宠物 NOUN
的 PART
家 NOUN
？ PUNCT


##### 依存关系解析

除了词性分析以外，我们还可以预测词与词之间的关系。比如一个词是某一个句子或者物体的主语。

`.dep_`属性返回预测的依存关系标注。

`.head`属性返回句法头词符。你可以认为这是词在句子中所依附的母词符。


In [30]:
for token in doc:
    print(token.text, token.pos_, token.dep_, token.head.text)

怎么 ADV advmod 解锁
解锁 VERB ROOT 解锁
宠物 NOUN nmod:assmod 家
的 PART case 宠物
家 NOUN dobj 解锁
？ PUNCT punct 解锁


##### 依存关系的定义

| Label     | Description | Example |
| --------- | ----------- | ------- |
| **nsubj** | 名词主语    |        |
| **dobj**  | 目的语      | 解锁  |

##### 命名实体识别

命名实体是那些被赋予了名字的真实世界的物体，比如一个人、一个组织或者一个国家。

从 `doc.ents` 中可以读取命名实体识别模型预测出的所有命名实体。

它会返回一个 `Span` 实例的遍历器，我们可以打印出实体文本和用 `.label_` 属性来打印出实体标注。

In [29]:
# 处理文本
doc = nlp("怎么解锁宠物的家？")

# 遍历识别出的实体
for ent in doc.ents:
    # 打印实体文本及其标注
    print(ent.text, ent.label_)

In [33]:
import spacy

# 确保已经安装了中文模型
# pip install spacy
# python -m spacy download zh_core_web_sm

# 加载中文模型
nlp = spacy.load("zh_core_web_sm")

# 使用包含命名实体的文本
doc = nlp("奥比岛在一个一望无际的大洋上。")

# 遍历识别出的实体
for ent in doc.ents:
    # 打印实体文本及其标注
    print(ent.text, ent.label_)


奥比岛 PERSON
大洋 LOC


In [37]:
spacy.explain("ADV")

'adverb'

##### 获得标注和标签定义

一个小诀窍是可以用 `spacy.explain` 这个帮手函数 来快速获得大部分常见的标注和标签定义。

举个例子，可能很多人不知道 "GPE" 代表的地理政治实体（geopolitical entity）的意思， 但调用 `spacy.explain` 我们就知道这是指国家、城市和州省。

同样这个方法也适用于词性标注和依存关系标注。

In [52]:
"""
怎么 ADV advmod 解锁
解锁 VERB ROOT 解锁
宠物 NOUN nmod:assmod 家
的 PART case 宠物
家 NOUN dobj 解锁
？ PUNCT punct 解锁
"""

# spacy.explain("ADV")
# spacy.explain("advmod")
# spacy.explain("VERB")
# spacy.explain("ROOT")
spacy.explain("assmod")

### 基于匹配的规则

本节课我们一起来学习spaCy的matcher， 用它来写一些规则来寻找文本中的目标词汇和短语。

为何不直接用正则表达式？

- 我们是在`Doc`对象中而不是直接在字符串上做匹配
- 我们是在词符及其属性中做匹配
- 我们会用到模型的预测结果
- 举个例子，"duck" (动词) vs. "duck" (名词)是不一样的（"duck"名词意思是鸭子，而动词是闪避的意思）

匹配的模板是一些列表，列表的每一个元素是一个字典。 每个字典代表一个词符，键值是词符属性名，映射到对应的目标值上面。

这个例子里我们要找两个文本为"iPhone"和"X"的词符。

我们也可以去匹配其它的词符属性。这里我们找两个小写形式为"iphone"和"x"的词符。

我们甚至可以直接调用模型的预测结果来写规则。这里我们找一个词根为"buy"且后面为名词的词符。 词根是词的基础形式，所以这个模板会匹配到诸如"buying milk"或者"bought flowers"这样的短语。

要使用模板我们首先从spacy.matcher中导入matcher。

我们还要读取一个流程创建nlp实例。

用模型分享出来的词汇表nlp.vocab来初始化matcher。 我们后面会详细介绍这一块，现在只要记得一定要传入这个词汇表就好了。

matcher.add方法可以用来添加一个模板。第一个参数是唯一的ID用来识别匹配的是哪一个模板。 第二个参数是一个模板的列表。

要在文本中匹配模板，我们可以在任何doc中调用matcher。

这样就会返回所有的匹配结果。

#### 模板匹配

- 一个元素是字典的列表，一个词符是一个元素
- 匹配词符的完全一致的文字

```
[{"TEXT": "iPhone"}, {"TEXT": "X"}]
```

- 匹配词汇属性

```
[{"LOWER": "iphone"}, {"LOWER": "x"}]
```

- 匹配任意的词符属性

```
[{"LEMMA": "buy"}, {"POS": "NOUN"}]
```

In [53]:
# 使用Matcher (1)：基于规则的匹配抽取

import spacy

# 导入Matcher
from spacy.matcher import Matcher

# 读取一个流程，创建nlp实例
nlp = spacy.load("zh_core_web_sm")

# 用模型分享出的vocab初始化matcher
matcher = Matcher(nlp.vocab)

# 给matcher加入模板
pattern = [{"TEXT": "iPhone"}, {"TEXT": "X"}]
matcher.add("IPHONE_PATTERN", [pattern])

# 处理文本
doc = nlp("即将上市的iPhone X发布日期被泄露了")

# 在doc上面调用matcher
matches = matcher(doc)

- `match_id`: 模板名的哈希值
- `start`: 匹配到的跨度的起始索引
- `end`: 匹配到的跨度的终止索引

In [55]:
# 使用Matcher (2)：基于规则的匹配抽取

# 在doc上调用matcher
doc = nlp("即将上市的iPhone X发布日期被泄露了")
matches = matcher(doc)

# 遍历所有的匹配结果
for match_id, start, end in matches:
    # 获取匹配的跨度
    matched_span = doc[start:end]
    print(matched_span.text)

iPhone X


##### 例子：模板匹配1

这是一个用到词汇属性的更复杂的匹配模板的例子。

我们要找五个词符：

一个只含有数字的词符；

三个匹配到"国际", "足联"和"世界杯"的词符；

以及一个标点符号词符。

这个模板最后可以匹配到"2018国际足联世界杯："。

In [61]:
import spacy

# 导入Matcher
from spacy.matcher import Matcher

# 读取一个流程，创建nlp实例
nlp = spacy.load("zh_core_web_sm")

# 用模型分享出的vocab初始化matcher
matcher = Matcher(nlp.vocab)

# 给matcher加入模板
pattern = [
    {"IS_DIGIT": True},
    {"LOWER": "国际"},
    {"LOWER": "足联"},
    {"LOWER": "世界杯"},
    {"IS_PUNCT": True}
]
matcher.add("IPHONE_PATTERN", [pattern])

# 处理文本
doc = nlp("2018国际足联世界杯：法国队赢了！")

# 在doc上面调用matcher
matches = matcher(doc)

# 遍历所有的匹配结果
for match_id, start, end in matches:
    # 获取匹配的跨度
    matched_span = doc[start:end]
    print(matched_span.text)

2018国际足联世界杯：


##### 例子：模板匹配2

这个例子中我们寻找两个词符：

一个词根是"喜欢"的动词，后面跟着一个名词。

这个模板最后可以匹配到"喜欢狗"和"喜欢猫"。

In [68]:
import spacy
from spacy.matcher import Matcher

# 读取一个流程，创建nlp实例
nlp = spacy.load("zh_core_web_sm")

# 用模型分享出的vocab初始化matcher
matcher = Matcher(nlp.vocab)

# 给matcher加入模板
pattern = [
    {"POS": "VERB"},
    {"POS": "NOUN"}
]
matcher.add("PATTERN_NAME", [pattern])

# 处理文本
doc = nlp("我喜欢狗但我更喜欢猫。")

# 在doc上面调用matcher
matches = matcher(doc)

# 遍历所有的匹配结果
for match_id, start, end in matches:
    # 获取匹配的跨度
    matched_span = doc[start:end]
    print(matched_span.text)

喜欢狗
喜欢猫


我们可以使用运算符和量词来定义一个词符应该被匹配几次。 我们可以用"OP"这个关键词来添加它们。

在这里"?"运算符使相应的判断词符变为可选， 所以我们会匹配到一个词根为"买"的词符，一个可选的数词和一个名词。

> 注意：中文分词"LEMMA"有错误，导致不能直接输出动词和名词形式的模板输出。问题待解决……

In [87]:
import spacy
from spacy.matcher import Matcher

# 读取一个流程，创建nlp实例
nlp = spacy.load("zh_core_web_sm")

# 用模型分享出的vocab初始化matcher
matcher = Matcher(nlp.vocab)

# 给matcher加入模板
pattern = [
    {"TEXT": "买"},
    {"POS": "NUM", "OP": "?"},  # 可选: 匹配0次或者1次
    {"POS": "NOUN"}
]
matcher.add("PATTERN_NAME", [pattern])

# 处理文本
doc = nlp("我买个肉夹馍。我还要买凉皮。")

# 在doc上面调用matcher
matches = matcher(doc)

# 遍历所有的匹配结果
for match_id, start, end in matches:
    # 获取匹配的跨度
    matched_span = doc[start:end]
    print(matched_span.text)

买个肉夹馍


"OP"可以有以下四种值：

"!"用来否定一个词符，所以它一次也不能被匹配。

"?"用来将一个词符变为可选，可以匹配0次或者1次。

"+"用来匹配目标词符1次或更多次。

最后，"*"用来匹配目标词符0次或更多次。

运算符可以大大加模板的威力，当然也带来了更多的复杂度。我们要学会善用它。



| 例子          | 说明               |
| ------------- | ------------------ |
| `{"OP": "!"}` | 否定: 0次匹配      |
| `{"OP": "?"}` | 可选: 0次或1次匹配 |
| `{"OP": "+"}` | 1次或更多次匹配    |
| `{"OP": "*"}` | 0次或更多次匹配    |